In [92]:
from ipywidgets import widgets
from IPython.display import display
import ipywidgets as widgets
import tensorflow as tf
import time
from IPython.display import display
import matplotlib.pyplot as plt
from matplotlib import image
#import cv2
from keras.preprocessing import image
import numpy as np

#config
IMG_SIZE = 224

import warnings
warnings.filterwarnings('ignore')


In [93]:
#custom loss function
def custom_loss_function(y_true, y_pred):
   imbalanced_cross_entropy = -(15 * tf.multiply(tf.cast(y_true, tf.float32), tf.math.log(y_pred))+tf.multiply(tf.cast((1 - y_true), tf.float32),tf.math.log(1- y_pred)))
   return tf.reduce_mean(imbalanced_cross_entropy, axis=-1)

#"area under the curve" as metric function
my_metric_fn = tf.keras.metrics.AUC(
    num_thresholds=200, curve='ROC',
    summation_method='interpolation', name=None, dtype=None,
    thresholds=None, multi_label=False, label_weights=None
    )

#preprocessing function for metadata
def preprocess_meta(sex,age,site):
    lis=[]
    for i in range(25):
        lis.append(0)
    if sex =='male':
        lis[0]=1
    integer= tf.cast(tf.floor((age/5)),tf.int64)
    lis[integer+1]=1
    numb = 6
    if site == 'head/neck':
        numb = 0
    if site =='upper extremity':
        numb = 1
    if site == 'lower extremity':
        numb = 2
    if site == 'torso':
        numb = 3
    if site == 'palms/soles':
        numb = 4
    if site == 'oral/genital':
        numb = 5
    for n in range(6):
        if n == numb:
            lis[19+numb]=1
    return tf.expand_dims(tf.convert_to_tensor(lis), axis=0)
    

#load image model
model = tf.keras.models.load_model('C:/Users/Flo/Documents/Python/TechLabs/Project/', custom_objects={'custom_loss_function': custom_loss_function, 'my_metric_fn': my_metric_fn})

model_1 = tf.keras.models.load_model('C:/Users/Flo/Documents/Python/TechLabs/Project/MetadatenModel', custom_objects={'custom_loss_function': custom_loss_function, 'my_metric_fn': my_metric_fn})


In [88]:


def page_one():
    global button1
    button1 = widgets.Button(description="start")
    global output1
    output1 = widgets.Output()
    display(button1, output1)
    button1.on_click(on_button_clicked_one)

def page_two():
    global int_range
    int_range = widgets.IntSlider(min=0, max=89, value=40, description='Your age: ')
    global radiobutton
    radiobutton = widgets.RadioButtons(
      options=['female', 'male'],
      value='female',
      description='Your sex: ',
      disabled=False
      )  
    global button2
    button2 = widgets.Button(description='next')
    global output2
    #radiobutton
    output2 = widgets.Output()
    display(int_range, radiobutton, button2, output2)
    #radiobutton.
    #observe(on_radiobutton)
    button2.on_click(on_button_clicked_two)

def page_three():
    global positionbar
    positionbar = widgets.Select(
        options=['None', 'head/neck', 'upper extremity', 'lower extremity', 'torso', 'palms/soles', 'oral/genital' ],
        value='None',
        description='Anatomic site: ',
        disabled=False
        )
    global uploadbutton
    uploadbutton = widgets.FileUpload(
        accept='.jpg',  # Accepted file extension e.g. '.txt', '.pdf', 'image/*', 'image/*,.pdf'
        multiple=False  # True to accept multiple files upload else False
        )
    
    global button3
    button3 = widgets.Button(description="next")
    global output3
    output3 = widgets.Output()
    display(positionbar, uploadbutton, button3, output3)
    button3.on_click(on_button_clicked_three)

def page_four():
    #save uploaded image
    img_name = uploadbutton.metadata[0]['name']
    content = uploadbutton.value[img_name]['content']
    with open('myfile.jpg', 'wb') as f: f.write(content)
        
    #make prediction with model - image
    img = image.load_img(('C:/Users/Flo/Documents/Python/TechLabs/Project/myfile.jpg') , target_size=(IMG_SIZE, IMG_SIZE))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    images = np.vstack([x])
    y_pred = model.predict(images, batch_size=None)[0][0]
    
    #make prediction with model - meta
    tensor = preprocess_meta(radiobutton.value, int_range.value, positionbar.value)
    z_pred = model_1.predict(tensor)[0][0]
    if z_pred < 0.45:
        z_suggestion = 'low risk'
    elif z_pred < 0.5:
        z_suggestion = 'increased risk'
    else:
        z_suggestion = 'higher risk'
    
    #print results
    global intro1
    intro1 = widgets.Label('Your results')
    
    result1 = widgets.Label(str(y_pred))
    result2 = widgets.Label(z_suggestion)
    
    global box1
    box1 = widgets.GridBox([widgets.Label('Image:'),widgets.Label('Meta'), result1, result2], layout=widgets.Layout(grid_template_columns="repeat(2, 300px)"))
    display(intro1)
    display(box1)
    
    #print data
    global intro2
    intro2 = widgets.Label('Your data')
    
    imagebox = widgets.Image(value=content, width=150, height=150 )
    sexLabel = widgets.Label('sex: '+str(radiobutton.value))
    ageLabel = widgets.Label('age: '+str(int_range.value))
    positionLabel = widgets.Label('position: '+ str(positionbar.value))
    
    global box2
    box2 = widgets.GridBox([imagebox, sexLabel, ageLabel, positionLabel], layout=widgets.Layout(grid_template_columns="repeat(4, 200px)"))
    display(intro2)
    display(box2)
    global button4
    button4 = widgets.Button(description="reset")
    global output4
    output4 = widgets.Output()
    display(button4, output4)
    button4.on_click(on_button_clicked_four)


def on_button_clicked_one(b):
    with output1:
        page_two()
        button1.close()

def on_button_clicked_two(b):
    with output2:
        page_three()
        button2.close()
        int_range.close()
        radiobutton.close()

def on_button_clicked_three(b):
    with output3:
        page_four()
        button3.close()
        uploadbutton.close()
        positionbar.close()

def on_button_clicked_four(b):
    with output4:
        intro1.close()
        box1.close()
        intro2.close()
        box2.close()
        button4.close()
        page_one()



page_one()

Button(description='start', style=ButtonStyle())

Output()